# Programmation côté serveur - 2ème partie

Dans cette deuxième partie, nous abordons différents thèmes:

- les **variables d'environnement** du programme cgi,

- enregistrer un **cookie** sur le navigateur ,

- exploiter un **formulaire** avec la méthode `GET`

Dans la troisième partie, nous aborderons:

- exploiter un **formulaire** avec la méthode `POST`; application à la réalisation d'une *session* très simple

- **persister les données** côtés serveur; application à l'enregistrement de messages d'un utilisateur identifié.

## Variables d'environnement - introduction

Tout programme en cours d'exécution (un *processus*), bénéficie d'un «contexte» qui consiste (entre autre) en différentes variables prépositionnées (par le système d'exploitation entre autre) dites **variables d'environnement**.

Pour afficher les variables d'environnement du shell:

```
$ printenv
USER=jupyter-etienne
LANG=fr_FR.UTF-8
OLDPWD=/home/jupyter-etienne
PWD=/home/jupyter-etienne/web
HOME=/home/jupyter-etienne
PATH=/home/jupyter-etienne/bin:/opt/tljh/user/bin:...
...
```


Dans un programme python, on peut accéder à ces variables en utilisant l'objet `environ` du module `os`; cet objet s'utilise *comme un dictionnaire*.

In [2]:
import os

path = os.environ["PATH"]
print(path)

/opt/tljh/user/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/snap/bin


Il est plus prudent d'utiliser la méthode `dict.get(clef, defaut)` pour se prémunir d'une erreur dans le cas où la clef n'existe pas.

In [3]:
mon_rep = os.environ.get("HOME", "~")
print(f"répertoire personnel: {mon_rep}")
mon_pid = os.environ.get("PID", "tiens, je pensais...")
print(f"pid: {mon_pid}")

répertoire personnel: /home/jupyter-etienne
pid: tiens, je pensais...


## Environnement produit par le serveur CGI

Un serveur web CGI, avant de lancer l'exécution d'un programme, renseigne son environnement avec des variables associées à la requête http qu'il a reçu.

En particulier, il renseigne les variables suivantes:

- `REQUEST_METHOD`: GET ou POST ?
- `QUERY_STRING`: le cas échéant, la chaîne de requête située après le `?` dans l'url,
- `HTTP_REFERER`: sur quelle page était le navigateur avant d'effectuer sa demande,
- `HTTP_COOKIE`: la chaîne brut du cookie précédemment enregistré (voir plus loin)
- ... et bien d'autres!

### Exemple

Le programme cgi <a href="demo_web_cgi/cgi-bin/environ.py">environ.py</a> (répertoire *demo_web_cgi/cgi-bin*) se propose d'afficher ces variables dans une page web.

### Vidéos d'aide

Désolé, j'ai été interrompu pendant la première ... la seconde est à la fin de la page.

In [4]:
%%HTML
<iframe src="https://player.vimeo.com/video/403640556" width="640" height="564" frameborder="0" allow="autoplay; fullscreen" allowfullscreen></iframe>

### Quelques rappels python

Remplacements dans un «gabarit» (*template* d'où le *_tpl* du code)

`gabarit.format(inser1=val1, inser2=val2, ...)`: renvoie la chaîne obtenue en remplaçant les zones d'insertions `{inser<n>}` du gabarit  par les valeurs fournis

In [ ]:
message_tpl = "blah blah {a} blah blah {b}".format(b="bli", a="blu")
message_tpl

Itérer (boucler sur) sur un dictionnaire avec `dict.items()`

In [ ]:
test = {"nom": "Doe", "prénom": "John"}
for k, v in test.items():
    print(
        "<li>{cle} => {valeur}</li>".format(
            cle=k,
            valeur=v
        )
    )

Construire une liste en compréhension `[expr for var in sequence]`

In [ ]:
li_tpl = "<li>{cle} => {valeur}</li>"
lis = [
    li_tpl.format(cle=k, valeur=v)
    for k, v in test.items()
]
lis

Former une chaine à partir d'une liste de chaînes `sep.join(liste_de_str)`

In [ ]:
print(
    '\n'.join(lis)
)

## Cookie

Un **cookie** est une petite donnée enregistrée par le navigateur à la demande du serveur.

En réponse à certaine circonstance, un serveur peut demander au navigateur (client) d'enregistrer une petite donnée en plaçant dans sa réponse un entête de la forme:

    Set-Cookie: <cle>=<val>; options

Le navigateur enregistre alors la donnée; après cela, il la retransmet dans l'entête de *chaque requête* (sur le même site) par la suite:

    Cookie: <cle1>=<val2>; <cle2>=<val2>; ...

Noter que le navigateur peut avoir plusieurs cookies pour le même site.

**À quoi cela sert-il?** on dit que le protocole http est un protocole *sans mémoire*: le serveur ne mémorise rien au sujet du client qui le consulte.

Pour que le serveur puisse réagir en fonction d'un client particulier, il est nécessaire que celui-ci transmette des informations qui permettent de le distinguer: c'est le rôle des cookies. 

### Exemple

Le programme cgi <a href="demo_web_cgi/cgi-bin/cookie.py">cookie.py</a> (répertoire *demo_web_cgi/cgi-bin*) illustre très basiquement l'utilisation de cookie pour savoir si une page a déjà été consultée.

## Formulaire géré par la méthode `GET`

Un formulaire html
```html
<form method="nom_meth" action="chemin_prog">
    <input name="champ1" .../>
    <input name="champ2" .../>
    ...
    <input type="submit"/>
</form>
```
permet à l'utilisateur d'envoyer des informations au serveur selon plusieurs méthodes de requête, en particulier `GET` ou `POST`.

La méthode choisie est placé dans l'attribut `method` du formulaire.

Lorsque l'utilisateur envoie (*submit*) le formulaire, cela donne lieue à une requête du navigateur.

Si la methode choisie est `GET`, le navigateur modifie l'**url** avec une **chaîne de requête** (qui suit le `?`) de la forme:

<div style="text-align: center;">
    <code>base_url/chemin_prog<strong>?champ1=val1&champ2=val2&...</strong></code>
</div>

*Note*: si l'attribut `action` n'est pas précisé, la requête est envoyée à la ressource en cours.

Depuis un programme python, on peut récupérer cette chaîne de requête  depuis `os.environ["QUERY_STRING"]` sous **forme brut** (*raw*)! Le programme n'a plus alors (!) qu'à extraire les champs et valeurs, effectuer les conversions nécessaires et à traiter le tout... autant vous dire que ce n'est pas facile du tout!

**heureusement**, Python nous fournit des outils pour faire tout cela simplement. Il propose un module standard `cgi` dont le but est de produire une sorte de dictionnaire - un `FieldStorage` - qui permet d'accéder simplement aux différents champs (*fields*) de la chaîne de requête. 

Basiquement, cela s'utilise de la façon suivante:

```python
import cgi

formulaire = cgi.FieldStorage()
val1 = formulaire.getvalue('champ1')
...
```

### Exemple

Le programme cgi <a href="demo_web_cgi/cgi-bin/form_get.py">form_get.py</a> (répertoire *demo_web_cgi/cgi-bin*) illustre très basiquement le traitement dynamique d'un formulaire soumis avec la méthode `GET`.

En testant l'exemple, modifier la chaîne de requête à la main pour voir ce que cela fait; observer aussi comment le navigateur encode le `#` de la couleur...

### Vidéo d'aide

In [5]:
%%HTML
<iframe src="https://player.vimeo.com/video/403633637" width="640" height="564" frameborder="0" allow="autoplay; fullscreen" allowfullscreen></iframe>